In [1]:
import pandas as pd
import json


In [2]:
# Read the JSON file into a DataFrame
with open('MyData/Playlist1.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract the desired columns
columns_to_extract = ['name', 'lastModifiedDate', 'trackName', 'artistName', 'albumName', 'trackUri', 'addedDate']
selected_data = []
for playlist in data['playlists']:
    for item in playlist['items']:
        row = [playlist['name'], playlist['lastModifiedDate'], item['track']['trackName'], item['track']['artistName'], item['track']['albumName'], item['track']['trackUri'], item['addedDate']]
        selected_data.append(row)

# Create a DataFrame from the selected data
df = pd.DataFrame(selected_data, columns=columns_to_extract)
df.columns = df.columns.str.lower()

# This is about my playlists

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up the Spotify API client
client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
data = []
for i in range(len(df)):
    track_uri = df.iloc[i]['trackuri']
    name_value = df.iloc[i]['name']

    # Spotipy API calling 
    track_info = sp.track(track_uri)
    track_features = sp.audio_features(track_uri)
    track_features = track_features[0]

    track_name = track_info['name']
    artist_name = track_info['artists'][0]['name']
    album_release_date = track_info['album']['release_date']
    track_popularity = track_info['popularity']

    if 'genres' in sp.artist(track_info['artists'][0]['id']):
        genres_list = sp.artist(track_info['artists'][0]['id'])['genres']
        if genres_list:
            genres = genres_list[0]
        else:
            # Handle the case when genres_list is empty
            genres = None
    else:
        # Handle the case when 'genres' key is missing in the response
        genres = None

    energy = track_features.get('energy')
    danceability = track_features.get('danceability')
    loudness = track_features.get('loudness')
    speechiness = track_features.get('speechiness')
    acousticness = track_features.get('acousticness')
    liveness = track_features.get('liveness')

    info = {
        'playlist': name_value, 
        'name': track_name,
        'artist': artist_name,
        'albumreleasedate': album_release_date,
        'genres': genres,
        'energy': energy,
        'danceability': danceability,
        'loudness': loudness,
        'speechiness': speechiness,
        'acousticness': acousticness,
        'liveness': liveness,
        'trackpopularity': track_popularity

    }
    data.append(info)
new_df = pd.DataFrame(data)
new_df.head()

,playlist,name,artist,albumreleasedate,genres,energy,danceability,loudness,speechiness,acousticness,liveness,trackpopularity
0,Descanso,Reminiscing,Dosi,2023-01-30,lo-fi study,0.309,0.550,-12.575,0.0410,0.911,0.0693,41
1,Descanso,miss you,Tibeauthetraveler,2021-08-14,lo-fi study,0.172,0.610,-14.957,0.0330,0.892,0.1010,54
2,Descanso,Escaped,Mujo,2021-01-08,japanese chillhop,0.206,0.783,-10.016,0.0663,0.906,0.1090,53
3,Descanso,Comfort,Bcalm,2021-02-25,focus beats,0.216,0.627,-15.371,0.0328,0.930,0.1220,52
4,Descanso,Ogmios,Cosmic Koala,2022-12-05,lo-fi study,0.270,0.478,-14.107,0.0468,0.486,0.1100,35


In [7]:
new_df.to_csv('output.csv')